<a href="https://colab.research.google.com/github/ccwbroomfield/NLP_work/blob/main/hate_speech_detection_and_write_up.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Classification for Hate Speech and Counterterrorism
Text classification, in which a natural language processing model predicts the class that a document belongs to based on past training data, has a wide variety of applications. Binary text classification is perhaps simplest, in which the document is considered to be one of two classes such as positive or negative sentiment, spam or ham, or library categorization. Text classification also applies to hate speech and counterterrorism, such as identifying social media posts for removal and account bans or determining the validity of a threat. 

In [ ]:
!pip install transformers datasets
#!pip install sentencepiece #needed this for second model

#paste model from huggingface

#https://huggingface.co/mrm8488/distilroberta-finetuned-tweets-hate-speech   #really sentiment analysis, trash
#from transformers import AutoTokenizer, AutoModelForSequenceClassification
#tokenizer = AutoTokenizer.from_pretrained("mrm8488/distilroberta-finetuned-tweets-hate-speech")
#model = AutoModelForSequenceClassification.from_pretrained("mrm8488/distilroberta-finetuned-tweets-hate-speech")

#https://huggingface.co/PaulAdversarial/T5_PAN_Hate_Speech_Twitter_topic_ishatespeach?text=is+this+hate+speech%3F  #trash
#from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
#tokenizer = AutoTokenizer.from_pretrained("PaulAdversarial/T5_PAN_Hate_Speech_Twitter_topic_ishatespeach")
#model = AutoModelForSeq2SeqLM.from_pretrained("PaulAdversarial/T5_PAN_Hate_Speech_Twitter_topic_ishatespeach")

#https://huggingface.co/Narrativa/byt5-base-tweet-hate-detection?text=%40user+black+lives+really+matter%3F   #sort of okay
#from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
#tokenizer = AutoTokenizer.from_pretrained("Narrativa/byt5-base-tweet-hate-detection")
#model = AutoModelForSeq2SeqLM.from_pretrained("Narrativa/byt5-base-tweet-hate-detection")

#https://huggingface.co/cardiffnlp/twitter-roberta-base-hate?text=i+hate+women
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-hate")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-hate")


In [ ]:
from transformers import pipeline
pipe = pipeline(model="cardiffnlp/twitter-roberta-base-hate")#, device=0)
#pass the text into pipe()
#can download everything from huggingface if you look for copy button
#tqdm is a progress bar, wrap the loop in tqdm()
#run on gpu for faster, but need to tell transformers to run on gpu and reimport everything. set device=0 in pipeline import
#gold[] has the tokens and tags?? zip to create list of tuples

Downloading:   0%|          | 0.00/935 [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset

#dataset = load_dataset("tweets_hate_speech_detection") #https://huggingface.co/datasets/tweets_hate_speech_detection
#dataset = load_dataset("hate_speech_offensive")       #https://huggingface.co/datasets/hate_speech_offensive
dataset = load_dataset("hate_speech18")          #https://huggingface.co/datasets/hate_speech18

Downloading:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/992 [00:00<?, ?B/s]

Using custom data configuration default


Downloading: 0.00B [00:00, ?B/s]

0 examples [00:00, ? examples/s]

Dataset hate_speech18 downloaded and prepared to /root/.cache/huggingface/datasets/hate_speech18/default/0.0.0/8033f254483a20d1d10b0f1b56ded1f54326b1617872c537f354a721a6951d47. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
#for first model, distilroberta
from tqdm import tqdm

def run_disrob(d):
  results = []

  for tweet in tqdm(d):   #go through all of the data
    result = pipe(tweet)
    if result[0]['label'] == 'NEGATIVE':   #given label and then score in list. so index 0 is label, get result. first model
      results.append(1)    #hate speech is negative, which is a 1 in the dataset
    else:
      results.append(0)
  
  return results


In [ ]:
#for second model, byt5
from tqdm import tqdm

def run_byt5(d):
  results = []

  for tweet in tqdm(d):   #go through all of the data
    result = pipe(tweet)
    if result[0]['generated_text'] == 'hate-speech':
      results.append(1)    #hate speech is negative, which is a 1 in the dataset
    else:
      results.append(0)
  
  return results

def classify_count(d):  #turn the counts of hate speech into yes or no, second dataset
  results = []
  for c in d:
    if(c > 0):
      results.append(1)
    else:
      results.append(0)
  return results

def hate_to_int(data):   #hate_speech18 dataset
  results = []
  for lab in data:
    if lab == 1:
      results.append(1)
    else:
      results.append(0)
  return results



In [ ]:
#for third model, twitter roberta
from tqdm import tqdm

def run_twit_rob(d):
  results = []

  for tweet in tqdm(d):   #go through all of the data
    result = pipe(tweet)
    if result[0]['label'] == 'LABEL_0':
      results.append(0)    #hate speech is negative, which is a 1 in the dataset
    else:
      results.append(1)
  
  return results

def classify_count(d):  #turn the counts of hate speech into yes or no, second dataset
  results = []
  for c in d:
    if(c > 0):
      results.append(1)
    else:
      results.append(0)
  return results

def hate_to_int(data):
  results = []
  for lab in data:
    if lab == 1:
      results.append(1)
    else:
      results.append(0)
  return results


#for d in dataset["train"]["label"][:15]:   #make sure i know how to get the text out of the first dataset again, and then figure out what pipe() looks like
 # print(d)
#print(run_twit_rob(dataset["train"]["tweet"][:100]))

#for d in dataset["train"]["class"][:10]:
#  print(d)

#for d in dataset["train"]["tweet"][:10]:
  #print(d)


In [ ]:
import datasets

accuracy = datasets.load_metric("accuracy")
precision = datasets.load_metric("precision")
recall = datasets.load_metric("recall")

#first model, first dataset
#results = run_disrob(dataset["train"]["tweet"][:5000])

#second model, second dataset
#results_model = run_byt5(dataset["train"]["tweet"][:4000])
#results_count = classify_count(dataset["train"]["hate_speech_count"][:4000])

#second model, third dataset
#results_model = run_byt5(dataset["train"]["text"][:6000]) 
#results_count = hate_to_int(dataset["train"]["label"][:6000])

#third model
results_model = run_twit_rob(dataset["train"]["text"][:5000])  #tweet for first two datasets, text for third
#first dataset
#results_count = dataset["train"]["label"][:5000]
#second dataset
#results_count = classify_count(dataset["train"]["hate_speech_count"][:5000])
#third dataset
results_count = hate_to_int(dataset["train"]["label"][:5000])




acc_class = accuracy.compute(predictions = results_model, references = results_count)
prec_class = precision.compute(predictions = results_model, references = results_count)
rec_class = recall.compute(predictions = results_model, references = results_count)

print("\naccuracy", acc_class)
print("precision", prec_class)
print("recall", rec_class)


  0%|          | 10/5000 [00:00<02:03, 40.54it/s]/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:978: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,
100%|██████████| 5000/5000 [00:48<00:00, 103.31it/s]



accuracy {'accuracy': 0.8808}
precision {'precision': 0.5351351351351351}
recall {'recall': 0.1625615763546798}


I ran the Narrativa/byt5-base-tweet-hate-detection model against the first 6000 tweets in the tweets_hate_speech_detection dataset.

*   Accuracy: .9666
*   Precision: .7175
*   Recall: .8416

This produced an F1 score of .7746. The model page claims an F1 score of 79.8, which seems within range depending on the sample used. 
<br/> This model was trained on the tweets_hate_speech_detection dataset, so I would expect it to be somewhat optimized for this dataset. 
<br>
To test the same model on a different dataset, I ran the model against the hate_speech_offensive dataset and the hate_speech18 datasets on the first 5000 and 4000 tweets, respectively.
<br/> The hate_speech_offensive dataset counts the number of instances of hate speech and offensive speech in a tweet. To modify to fit the model I considered any tweet with 1 or more counts of hate speech to be hate speech. The model performed very poorly on this dataset:

*   Accuracy: 0.5533
*   Precision: 0.2574
*   Recall: 0.5419

<br/>The hate_speech18 dataset labels data as no hate speech, hate speech, or hate speech only in combination with other text. This model also did not appear to be very strong:

*   Accuracy: 0.6657
*   Precision: 0.2201
*   Recall: 0.6447
<br/>These datsets give an F1 score of 0.349 and 0.328, respectively. I was surprised that the hate_speech_offensive_dataset had a higher F1 score associated with it.
<br/> Some of this may be due to optimization of the model on the tweets_hate_speech_detection dataset, as the model has about twice as high an F1 score on the dataset it was trained on compared to any other dataset.
<br/>I think part of this may be due to labelling. Defining hate speech is tricky, and I think all of these datasets take some liberties in labelling. Looking through the tweets_hate_speech_detection dataset specifically I felt that several pieces of text did not make sense as hate speech. Lacking context, such as the images or videos attached to the text, further complicates the problem. I imagine this makes it extremely challenging to create a generalizeable model from any one collection. 





I also ran the mrm8488/distilroberta-finetuned-tweets-hate-speech on the tweets_hate_speech_detection dataset, for which the model claimed it was fine tuned. 


*   Accuracy: 0.4948
*   Precision: 0.1081
*   Recall: 0.8550

<br> The model claims a 98% validation accuracy, but provides no other information. However, on analyzing the pipe() results, the model was returning positive or negative as if it was really performing sentiment analysis. This meant that many things were being labelled as negative sentiment, perhaps correctly, but had very little bearing on whether the text was hate speech or not. I did not continue use of this model due to these shortcomings.



Finally I ran the twitter-roberta-base-hate model against the tweets_hate_speech_detection dataset with the following results:

*   Accuracy: 0.9262
*   Precision: 0.25
*   Recall: 0.04

On the hate_speech_offensive dataset with the following results:

*   Accuracy: 0.5704
*   Precision: 0.1813
*   Recall: 0.247

On the hate_speech18 dataset with the following resutls:


*   Accuracy: 0.8808
*   Precision: 0.5351
*   Recall: 0.1626


In my own testing I discovered that this model was far pickier about what was considered hate speech. So while the metrics on the datasets with this model were generally dismal, I'm somewhat inclined to believe that this reflects the liberties taken with labelling the 3 datasets I used rather than a fault of the model. For example, "retweet if you agree" was marked as hate speech in the tweets_hate_speech_detection dataset. Any model that classifies this as hate speech without additional context would be manfunctioning. The model claims that it was trained on 58 million tweets and further fine-tuned, but this dataset does not appear to be available online. Either way, this model may serve as more of an authority than the labelling on my datasets (however that was done). This was the first time where I've real doubts conerning the training data, and this made the issue of having good well-labelled data much more tangible.



# Dataset Concerns:

As with any text classification problem, access to high quality and well-labelled data is a necessity. This is a challenge for hate-speech classification, as definitions of hate speech vary. Using crowd-sourced data or removed posts is not always reliable, and the text itself is not useful if the text requires additional context to be considered hate speech.
<br> I also expected that models published on huggingface would be good. Thy were not, and many seemed to not do what they said they did. I was surprised by the low-quality of so many of them.
<br> Perhaps more concerning is how little information exists on the dataset. Very little information is given on how the data was collected or curated, and no mention is made to the well-documented bias in dialects and other characteristics for most of the datasets I looked like. I felt that the labelling was unreliable, and the actual fields sometimes differed between the data preview and the downloaded dataset (such as representing text classes with integers without a clear mapping from one to the other). This made it clear that there are huge differences between a dataset and a useful dataset. These concerns lead me to believe that the third model I tested may be more accurate than the annotations in the training data. This mistrust of the datasets makes them somewhat useless.